In [1]:
import xml.etree.ElementTree as ET
import pickle
import os
from os import listdir, getcwd
from os.path import join
from PIL import Image

sets=['train','val']

classes = ["face_mask","face"]

def convert(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

def convert_annotation(setType, imageID):
    flag=0
    in_file = open('FaceMaskDataset/%s/Annotations/%s.xml'%(setType, image_id))
    
    tree=ET.parse(in_file)
    root = tree.getroot()
    
    size = root.find('size')
    
    if size==None or int(size.find('width').text)==0 or int(size.find('height').text)==0:
        img = Image.open('FaceMaskDataset/%s/Images/%s.jpg'%(setType, image_id))
        w=img.size[0]
        h=img.size[1]
    else:
        w = int(size.find('width').text)
        h = int(size.find('height').text)
        
    writeContent=""

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult) == 1:
            continue
       
        flag=1
        cls_id = classes.index(cls)+1
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        writeContent=writeContent+str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n'

    if flag==1:
        out_file = open('FaceMaskDataset/%s/Images/%s.txt'%(setType, image_id), 'w')
        out_file.write(writeContent)
        
    return flag

        
def getFileID(file_dir):   
    L=[]   
    for file in os.listdir(file_dir):
        fileNameM,ext=os.path.splitext(file)
        if ext==".jpg":
            L.append(fileNameM)
    return L  
        
        
wd = getcwd()

for setType in sets:
    if not os.path.exists('FaceMaskDataset/%s/labels/'%(setType)):
        os.makedirs('FaceMaskDataset/%s/labels/'%(setType))
    image_ids = getFileID('FaceMaskDataset/%s/Images/'%(setType))
    list_file = open('%s.txt'%(setType), 'w')
    for image_id in image_ids:
        flag=convert_annotation(setType, image_id)
        if flag==1:
            name=wd+'/FaceMaskDataset/'+setType+'/Images/'+image_id+'.jpg\n'
            list_file.write(name)
    list_file.close()





In [20]:
import os
from PIL import Image

img = Image.open('FaceMaskDataset/train/Images/1_Handshaking_Handshaking_1_42.jpg')

print(img.format)       # PNG
print(img.size[0])          # (3500, 3500)


JPEG
1024


In [2]:
classes

['face_mask', 'face', 'face_nask']

In [ ]:
import os  
def file_name(file_dir):   
    L=[]   
    for root, dirs, files in os.walk(file_dir):  
        for file in files:  
            if os.path.splitext(file)[1] == '.jpeg':  
                L.append(os.path.join(root, file))  
    return L  